####Note

- The main function `readExpData()` is at the bottom. Please ensure that all of code above `readExpData()` has been compiled and run before launching the main function.
- Just compile and run sequentially.

In [ ]:
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as functional
import torchvision 
from torchvision.datasets import ImageFolder
import os
from xlwt import Workbook

In [ ]:
custom_transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.5 ), (0.5))],
    )
classes = ('0', '1', '2', '3', '4', '5', '6', '7', '8', '9')

In [ ]:
class Network(nn.Module):
    def __init__(self):
        self.output_size = 10
        
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2)
        self.fc = nn.Linear(400, 120)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, self.output_size)
        
    def forward(self, x):
        x = self.pool(functional.relu(self.conv1(x)))
        x = self.pool(functional.relu(self.conv2(x)))
        x = x.view(-1, 400)
        x = functional.relu(self.fc(x))
        x = functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def processImage(img):
    ned = cv2.imread("./E.png")
    result = cv2.matchTemplate(img, ned, cv2.TM_CCOEFF_NORMED)
    _, _, _, max_loc = cv2.minMaxLoc(result)
    test_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, test_img = cv2.threshold(test_img, 150, 255, cv2.THRESH_BINARY)

    sample1 = test_img[max_loc[1] : max_loc[1]+21, max_loc[0]-41 : max_loc[0]-20]
    sample1 = cv2.resize(sample1,(32, 32))
    sample2 = test_img[max_loc[1] : max_loc[1]+21, max_loc[0]-19 : max_loc[0]+1]
    sample2 = cv2.resize(sample2, (32, 32))
    sample3 = test_img[max_loc[1] : max_loc[1]+21, max_loc[0]+36 : max_loc[0]+57]
    sample3 = cv2.resize(sample3, (32, 32))
    return sample1, sample2, sample3

In [ ]:
def check2(image, net):#test three images
    test_path1 = './1/'
    test_path = './1/1/'
    first_path = './1/1/first.png'
    second_path = './1/1/second.png'
    third_path = './1/1/third.png'
    show_tag = False
    first, second, third = processImage(image)
    if os.path.exists(test_path1):
        if os.path.isfile(first_path):
            os.remove(first_path)
        if os.path.isfile(second_path):
            os.remove(second_path)
        if os.path.isfile(third_path):
            os.remove(third_path)
        if os.path.exists(test_path):
            os.removedirs(test_path)
        
    os.makedirs(test_path)
    first = cv2.resize(first, (32, 32))
    cv2.imwrite(first_path, first)
    second = cv2.resize(second, (32, 32))
    cv2.imwrite(second_path, second)
    third = cv2.resize(third, (32, 32))
    cv2.imwrite(third_path, third)
    if show_tag:
        plt.imshow(first)
        plt.show()
        plt.imshow(second)
        plt.show()
        plt.imshow(third)
        plt.show()
    real_test = ImageFolder(root = test_path1, transform = custom_transform)
    real_testloader = torch.utils.data.DataLoader(real_test,
        batch_size = 3,
        shuffle = False)
    dataiter = iter(real_testloader)
    images, labels = dataiter.next()
    outputs = net(images)
    sm = nn.Softmax(dim=1)      
    sm_outputs = sm(outputs)
    probs, index = torch.max(sm_outputs, dim=1)
    first = eval(classes[index[0]])
    second = eval(classes[index[1]])
    third = eval(classes[index[2]])
    # print(first)
    # print(second)
    # print(third)
    result = (first + second*0.1)*(10**third)
    os.remove(first_path)
    os.remove(second_path)
    os.remove(third_path)
    os.removedirs(test_path)
    return result

In [ ]:
#assist functions of `ReadExpData()`
def loadNet(netpath):
    net = Network()
    net.load_state_dict(torch.load(netpath))
    return net

def openVideo(videoname):
    video = cv2.VideoCapture("./" + videoname)
    #check if the video exists
    if not video.isOpened():
        raise Exception("Video cannot be opened.")
    return video

def showVideoInfo(videoname, video):
    #show some info of video
    print("Video: " + videoname)
    print("Total frams: " + str(video.get(7)))
    print("FPS: " + str(video.get(5)))
    #if sampling_rate = 10, it means every 0.1s we capture a photo
    sampling_rate = 10
    print("Sampling_rate: " + str(sampling_rate) + "Hz") 
    # timeF: we pick one frame every 'timeF' frames.
    # Here we pick one frame every 5 frames. 
    timeF = video.get(5) / sampling_rate 
    print("Frams needed to be extracted:" + str(video.get(7)/timeF))
    return sampling_rate, timeF

def initExcelTable():
    #create and initialize an Excel table
    wb = Workbook()
    sheet1 = wb.add_sheet('Sheet 1')
    sheet1.write(0, 0, "Time/s")
    sheet1.write(0, 1, "Pressure/Pa")
    print("Time/s", "Pressure/Pa")
    return wb, sheet1

def saveResult(wb, videoname):
    if os.path.exists(videoname+"_result/"):
        os.removedirs(videoname+"_result/")    
    os.makedirs(videoname+"_result/")
    wb.save(videoname+"_result/"+'result.xls')

In [ ]:
def readExpData(_videoname, _netpath = './result.pth'):
    # load the trained convolution network
    net = loadNet(_netpath)
    for videoname in _videoname:
        video = openVideo(videoname)
        # timeF: we pick one frame every 'timeF' frames.
        # Here we pick one frame every 5 frames. 
        _, timeF = showVideoInfo(videoname, video) 
        wb, sheet1 = initExcelTable()
        rval = True
        frame_cnt = 1
        result_cnt = 1
        while rval:  
            # Keep reading frames until rval=False(that is, end of file)
            rval, frame = video.read()
            if (frame_cnt % timeF == 0 and rval): 
                # take down the data
                result = check2(frame, net)
                print(result, result_cnt) #for debug
                sheet1.write(result_cnt, 1, result)
                sheet1.write(result_cnt, 0, result_cnt/10) #'result_cnt/10' means time(units of sec)
                result_cnt += 1
            frame_cnt += 1
        #save the excel table
        saveResult(wb, videoname)

####Main function
- parameters:
  - _videoname: a list contains all names of video you want to extract experiment data. Note that all video should be at the same directory as the program.
  - _netpath: the name of file of the network you previously train.

In [ ]:
#Main function
#for each video, you may change the variable 'videoname'
readExpData(_videoname = ["35_1.MTS", "35_2.MTS"], _netpath="./result_bsize=24, epoch=100.pth")